Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Using Paramater Efficient Fine Tuning on Llama 2 with 7B Parameters on One Intel&reg; Gaudi&reg; 2 AI Accelerator
This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuining (PEFT) and then run inference on a text prompt.  This will be using the Llama2 model with two task examples from the Optimum Habana library on the Hugging Face model repository.   The Optimum Habana library is optimized for Deep Learning training and inference on First-gen Gaudi and Gaudi2 and offers tasks such as text generation, language modeling, question answering and more. For all the examples and models, please refer to the [Optimum Habana GitHub](https://github.com/huggingface/optimum-habana#validated-models).

This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuining (PEFT) on the timdettmers/openassistant-guanaco dataset using the Language-Modeling Task in Optimum Habana.

> **Note:** This notebook and hosted environment have been designed and optimized for Intel® Gaudi® software version 1.15. Please keep this in mind if attempting to run other notebooks or activities, as content designed for newer software versions is not likely to run successfully. We will be upgrading this version soon. Thanks!

### Parameter Efficient Fine Tuning with Low Rank Adaptation
Parameter Efficient Fine Tuning is a strategy for adapting large pre-trained language models to specific tasks while minimizing computational and memory demands.   It aims to reduce the computational cost and memory requirements associated with fine-tuning large models while maintaining or even improving their performance.  It does so by adding a smaller task-specific layer, leveraging knowledge distillation, and often relying on few-shot learning, resulting in efficient yet effective models for various natural language understanding tasks.   PEFT starts with a pre-trained language model that has already learned a wide range of language understanding tasks from a large corpus of text data. These models are usually large and computationally expensive.   Instead of fine-tuning the entire pre-trained model, PEFT adds a task-specific layer or a few task-specific layers on top of the pre-trained model. These additional layers are relatively smaller and have fewer parameters compared to the base model.


In [1]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials

/home/uc23ff5572e2e4b3554d551d65d833aa/Gaudi-tutorials/PyTorch/Single_card_tutorials


### Model Setup: 

##### Install the Parameter Efficient Fine Tuning Library methods
This is taking the PEFT method from the Hugging Face repository and will be used to help create the PEFT Fine Tuning with the Llama2 model.

In [2]:
import sys
!{sys.executable} -m pip install peft==0.10.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Attempting uninstall: peft
    Found existing installation: peft 0.6.2
    Uninstalling peft-0.6.2:
      Successfully uninstalled peft-0.6.2


##### Install the Optimum-Habana Library

In [3]:
!{sys.executable} -m pip install -q optimum-habana==1.11.1

##### Pull the Hugging Face Examples from GitHub
These contain the working Hugging Face Task Examples that have been optimized for Gaudi.  For Fine Tuning, we'll use the language-modeling task. 

In [4]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials

/home/uc23ff5572e2e4b3554d551d65d833aa/Gaudi-tutorials/PyTorch/Single_card_tutorials


In [5]:
!git clone -b v1.11.1 https://github.com/huggingface/optimum-habana.git

fatal: destination path 'optimum-habana' already exists and is not an empty directory.


##### Go to the Language Modeling Task and install the model specific requirements

In [6]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling/ 


/home/uc23ff5572e2e4b3554d551d65d833aa/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling


In [7]:
! python3.10 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install -q -r requirements.txt

##### How to access and Use the Llama 2 model

Use of the pretrained model is subject to compliance with third party licenses, including the “Llama 2 Community License Agreement” (LLAMAV2). For guidance on the intended use of the LLAMA2 model, what will be considered misuse and out-of-scope uses, who are the intended users and additional terms please review and read the instructions in this link https://ai.meta.com/llama/license/.
Users bear sole liability and responsibility to follow and comply with any third party licenses, and Habana Labs disclaims and will bear no liability with respect to users’ use or compliance with third party licenses.

To be able to run gated models like the Meta Llama model, you need the following: 
- Have a HuggingFace account
- Agree to the terms of use of the model in its model card on the HF Hub
- set a read token
- Login to your account using the HF CLI: run huggingface-cli login before launching your script

In [ ]:
!huggingface-cli login --token #your token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `gaudi-token-sample` has been saved to /home/uc23ff5572e2e4b3554d551d65d833aa/.cache/huggingface/stored_tokens
Your token has been saved to /home/uc23ff5572e2e4b3554d551d65d833aa/.cache/huggingface/token
Login successful.
The current active token is: `gaudi-token-sample`


## Fine Tuning the model with PEFT and LoRA

We'll now run the fine tuning with the PEFT method. Remember that the PEFT methods only fine-tune a small number of extra model parameters, thereby greatly decreasing the computational and storage costs. Recent State-of-the-Art PEFT techniques achieve performance comparable to that of full fine-tuning.

##### Here's a summary of the command required to run the Fine Tuning, you'll run this in the next cell below. 
Note in this case the following: 
1. Using the language modeling with LoRA; `run_lora_clm.py`
2. It's very efficient: only 0.06% of the total paramters are being fine tuned of the total 7B parameters.
4. Only 3 epochs are needed for fine tuning, it takes less than 20 minutes to run with the openassisant-guanaco dataset.
5. We are using a cached model on the Intel Tiber Cloud to reduce download time.


In [10]:
!pip install habana-frameworks --extra-index-url https://vault.habana.ai/artifactory/api/pypi/gaudi-python/simple


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://vault.habana.ai/artifactory/api/pypi/gaudi-python/simple
ERROR: Could not find a version that satisfies the requirement habana-frameworks (from versions: none)
ERROR: No matching distribution found for habana-frameworks


In [11]:
!python3 run_lora_clm.py \
    --overwrite_output_dir=True \
    --model_name_or_path VAGOsolutions/Llama-3-SauerkrautLM-70b-Instruct \
    --dataset_name timdettmers/openassistant-guanaco \
    --bf16 True \
    --output_dir ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/2025 \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --evaluation_strategy "no" \
    --save_strategy "no" \
    --learning_rate 1e-4 \
    --warmup_ratio  0.03 \
    --lr_scheduler_type "constant" \
    --max_grad_norm  0.3 \
    --logging_steps 1 \
    --do_train \
    --do_eval \
    --use_habana \
    --use_lazy_mode \
    --throughput_warmup_steps 3 \
    --lora_rank=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --lora_target_modules "q_proj" "v_proj" \
    --dataset_concatenation \
    --report_to none \
    --max_seq_length 512 \
    --trust_remote_code=True \
    --low_cpu_mem_usage True \
    --validation_split_percentage 4 \
    --adam_epsilon 1e-08

Traceback (most recent call last):
  File "/home/uc23ff5572e2e4b3554d551d65d833aa/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling/run_lora_clm.py", line 754, in <module>
    main()
  File "/home/uc23ff5572e2e4b3554d551d65d833aa/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling/run_lora_clm.py", line 365, in main
    ) = parser.parse_args_into_dataclasses()
  File "/home/uc23ff5572e2e4b3554d551d65d833aa/.local/lib/python3.10/site-packages/transformers/hf_argparser.py", line 338, in parse_args_into_dataclasses
    obj = dtype(**inputs)
  File "<string>", line 140, in __init__
  File "/home/uc23ff5572e2e4b3554d551d65d833aa/.local/lib/python3.10/site-packages/optimum/habana/transformers/training_args.py", line 640, in __post_init__
    device_is_hpu = self.device.type == "hpu"
  File "/home/uc23ff5572e2e4b3554d551d65d833aa/.local/lib/python3.10/site-packages/transformers/training_args.py", line 1995, in device
  

Defaulting to user installation because normal site-packages is not writeable
  Using cached flash_attn-2.7.4.post1.tar.gz (6.0 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      /tmp/pip-install-sexhsps3/flash-attn_2e99c4b4162d46c38025ed80eb1f65e5/setup.py:106: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-sexhsps3/flash-attn_2e99c4b4162d46c38025ed80eb1f65e5/setup.py", line 198, in <module>
          CUDAExtension(
        File "/usr/loca

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://github.com/Dao-AILab/flash-attention/releases
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.8/187.8 MB 75.4 MB/s eta 0:00:0000:0100:01
Discarding https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.4.post1/flash_attn-2.7.4.post1%2Bcu12torch2.3cxx11abiFALSE-cp310-cp310-linux_x86_64.whl (from https://github.com/Dao-AILab/flash-attention/releases): Requested flash-attn from https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.4.post1/flash_attn-2.7.4.post1%2Bcu12torch2.3cxx11abiFALSE-cp310-cp310-linux_x86_64.whl has inconsistent version: expected '2.7.4.post1+cu12torch2.3cxx11abifalse', but metadata has '2.7.4.post1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 MB 28.8 MB/s eta 0:00:0000:0100:01
Discarding https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.4.post1/flash_attn-2.7.4.post1%2Bcu12torch2.2cxx11abiTRUE-cp310-cp310-l

#### LoRA Fine Tuning Completed
You will now see a "model_lora_llama_single" folder created which contains the PEFT model `adapter_model.bin` which will be used in the inference example below. 

## Inference with Llama 2

We'll now use the Hugging Face `text-generation` task to run inference on the Llama2-7b model; we'll generate text based on an included prompt.  Notice that we've included a path to the PEFT model that we've just created.

First, we'll move to the text-generation examples folder and install the requirements. 

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/text-generation
!pip install -q -r requirements.txt

You will see that we are now running inference with the `run_generation.py` task and we are including the PEFT model that we Fine Tuned in the steps above. 

```
--prompt "I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!" \
--peft_model /root/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling/model_lora_llama_single
```

In [ ]:
# Use the prompt above to compare between the PEFT and non-PEFT examples
prompt = input("Enter a prompt for text generation: ")

In [ ]:
cmd = f'python3 run_generation.py  --model_name_or_path VAGOsolutions/Llama-3-SauerkrautLM-70b-Instruct --batch_size 1 --do_sample --max_new_tokens 300 --n_iterations 4 \
      --use_hpu_graphs --use_kv_cache --bf16 --prompt "{prompt}" \
      --peft_model ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/model_one '
print(cmd)
import os
os.system(cmd)

##### Comparison without PEFT and LoRA
In this example, we're simply running the Llama2 7B model **without** including the PEFT fine tuned model, so the you are losing the additional detail that is brought to the model, and the results have signficantly less information and fidelity compared to the last model.

In [ ]:
cmd = f'python3 run_generation.py  --model_name_or_path VAGOsolutions/Llama-3-SauerkrautLM-70b-Instruct --batch_size 1 --do_sample --max_new_tokens 300 --n_iterations 4 \
      --use_hpu_graphs --use_kv_cache --bf16 --prompt "{prompt}"'
print(cmd)
import os
os.system(cmd)

In [ ]:
# Please be sure to run this exit command to ensure that the resorces running on Intel Gaudi are released 
exit()